In [1]:
import numpy as np
import ROOT as rt

import matplotlib.pyplot as plt
%matplotlib notebook
plt.style.use('olmo')

from histo_utilities import create_TH1D
from cebefo_style import cebefo_style
cebefo_style()

c_light = 2.99792458E8 #m/s

!cd /Users/olmo/programs/Delphes-3.4.1
rt.gSystem.Load("libDelphes");
rt.gInterpreter.Declare('#include "classes/DelphesClasses.h"');
rt.gInterpreter.Declare('#include "external/ExRootAnalysis/ExRootTreeReader.h"');

Welcome to JupyROOT 6.10/08


In [2]:
file_path = ['/Users/olmo/cernbox/PID_timing_studies/_root/gg2Rhad_PU10_trial.root']

chain = rt.TChain('Delphes')
for path in file_path:
    chain.Add(path)
    
# chain.Print()

treeReader = rt.ExRootTreeReader(chain)
numberOfEntries = treeReader.GetEntries()

branches_names = ['Event', 'Particle', 'Track','Vertex4D', 'GenVertex']
branch = {}

for b in branches_names:
    branch[b] = treeReader.UseBranch(b)

ExRootAnalysisDict dictionary payload:16:10: fatal error: 'ExRootAnalysis/ExRootTreeReader.h' file not found
#include "ExRootAnalysis/ExRootTreeReader.h"
         ^
Error in <TInterpreter::AutoParse>: Error parsing payload code for class ExRootTreeReader with content:

#line 1 "ExRootAnalysisDict dictionary payload"

#ifndef G__VECTOR_HAS_CLASS_ITERATOR
  #define G__VECTOR_HAS_CLASS_ITERATOR 1
#endif

#define _BACKWARD_BACKWARD_WARNING_H

/** \class ExRootAnalysisLinkDef
 *
 *  Lists classes to be included in cint dicitonary
 *
 *  \author P. Demin - UCL, Louvain-la-Neuve
 *
 */

#include "ExRootAnalysis/ExRootTreeReader.h"
#include "ExRootAnalysis/ExRootTreeWriter.h"
#include "ExRootAnalysis/ExRootTreeBranch.h"
#include "ExRootAnalysis/ExRootResult.h"
#include "ExRootAnalysis/ExRootUtilities.h"
#include "ExRootAnalysis/ExRootClassifier.h"
#include "ExRootAnalysis/ExRootFilter.h"

#include "ExRootAnalysis/ExRootProgressBar.h"
#include "ExRootAnalysis/ExRootConfReader.h"
#include "ExRoo

In [7]:
for i in range(numberOfEntries):
    treeReader.ReadEntry(i)
    nv = branch['Vertex4D'].GetEntries()
    
    sumpt2_reco = np.zeros(nv)
    for j in range(nv):
        sumpt2_reco[j] = branch['Vertex4D'].At(j).SumPT

    sumpt2_REreco = np.zeros(nv)
    for j in range(branch['Track'].GetEntries()):
        track = branch['Track'].At(j)
        if track.VertexIndex >= 0:
            sumpt2_REreco[track.VertexIndex] += track.PT
            
    if np.sum(np.abs(sumpt2_reco - sumpt2_REreco))>1:
        print "-----  Event", i
        for j in range(nv):
            print '{}: {:.1f}  -  {:.1f}'.format(j, sumpt2_REreco[j] , sumpt2_reco[j])

In [4]:
# gvtx_names = ['Nev','z','t', 'IsPU']
# gvtx_att = []

# rvtx_names = ['Nev','z','t', 'dz', 'dt', 'ndf', 'sumpt2', 'sum_pt', 'N_Rhad', 'max_pt_tk']
# rvtx_att_global = []


# for i in range(numberOfEntries):
#     treeReader.ReadEntry(i)
    
#     for j in range(branch['GenVertex'].GetEntries()):
#         vtx = branch['GenVertex'].At(j)
        
#         p = vtx.Constituents.At(0)
#         ispu = p.IsPU
        
#         gvtx_att.append((i, vtx.Z, vtx.T*1E12, ispu))
    
#     rvtx_att = []
#     for j in range(branch['Vertex4D'].GetEntries()):
#         vtx = branch['Vertex4D'].At(j)
        
#         rvtx_att.append([i, vtx.Z, vtx.T*1E12, vtx.ErrorZ, vtx.ErrorT*1E12, vtx.NDF, 0, 0, 0, 0])
        
#     for j in range(branch['Track'].GetEntries()):
#         track = branch['Track'].At(j)
#         if track.VertexIndex >= 0:
#             k = track.VertexIndex
#             pt = track.PT
#             rvtx_att[k][6] += pt**2
#             rvtx_att[k][7] += pt
            
#             if pt>rvtx_att[k][9]:
#                 rvtx_att[k][9] = pt
            
#             if abs(track.PID)>5000:
#                 rvtx_att[k][8] += 1
                
#     for j in range(len(rvtx_att)):
#         rvtx_att_global.append(rvtx_att[j])
        
# #         np = vtx.Constituents.GetEntriesFast()
# #         for k in range(np):
# #             tr = vtx.Constituents.At(k)
# #             print tr.GetName()
    

# gen_vtx = np.array(gvtx_att, dtype=zip(gvtx_names,['<f8']*len(gvtx_names)))
# reco_vtx = np.array(rvtx_att_global, dtype=zip(rvtx_names,['<f8']*len(rvtx_names)))

# print reco_vtx.shape

Qualche plot per far vedere la qualita dei reco vtx

In [5]:
# %matplotlib notebook
# N_ev = 15

# plt.figure(figsize=(8,8))

# z_gen = gen_vtx['z'][gen_vtx['Nev'] == N_ev]
# t_gen = gen_vtx['t'][gen_vtx['Nev'] == N_ev]


# z_reco = reco_vtx['z'][reco_vtx['Nev'] == N_ev]
# t_reco = reco_vtx['t'][reco_vtx['Nev'] == N_ev]
# dz_reco = reco_vtx['dz'][reco_vtx['Nev'] == N_ev]
# dt_reco = reco_vtx['dt'][reco_vtx['Nev'] == N_ev]

# # the PV is always the first
# plt.plot(t_gen[0], z_gen[0], 'Dg', label='Gen PV', markersize=7)
# plt.plot(t_gen[1:], z_gen[1:], '*g', label='Gen PU', markersize=5)


# plt.errorbar(t_reco, z_reco, xerr=dt_reco, yerr=dz_reco, fmt='r.', label='Reco Vtx', markersize=7)

# plt.legend(loc='best')
# plt.xlabel(r'$Time$  [ps]')
# plt.ylabel(r'$Z$ [mm]')
# plt.grid()

In [6]:
# for i in range(numberOfEntries):
#     treeReader.ReadEntry(i)
#     nv = branch['Vertex4D'].GetEntries()
#     sumpt2_reco = np.zeros(nv)
#     for j in range(nv):
#         vtx = branch['Vertex4D'].At(j)
#         sumpt2_reco[j] = vtx.SumPT2
    
#     sumpt2_REreco = np.zeros(nv)
#     for j in range(branch['Track'].GetEntries()):
#         track = branch['Track'].At(j)
#         if track.VertexIndex >= 0:
#             sumpt2_REreco[track.VertexIndex] += track.PT**2
#     if np.sum(np.abs(sumpt2_reco - sumpt2_REreco))>1:
#         print i,':', sumpt2_reco, sumpt2_REreco
#         print i,':', sumpt2_reco - sumpt2_REreco